## **5.1 계산 그래프**

- 계산 그래프 노드와 에지 노드는 + - 표기 에지 -> 표기

- 100 * 2 * 1.1 = 220 
- (100) -> (2) -> node(*) -> (200) -> 1.1 -> node(*)-> (220) 
- 순전파 forward propagation : 계산이 왼쪽에서 오른쪽으로 진행 
- 역전파 backward propagation : 순전파와 반대로 오른쪽에서 왼쪽으로 진행
- 국소적 계산 : 자신과 직접 관련된 것들을 넘겨 준다 
- 예시 : node(*)는 두 숫자를 곱한다라는 국소적 계산을 실시 
- 단순한 국소적 계산이 모여 복잡한 계산 구현 가능 

## 그림 5.5

- 역전파인 경우 1 -> 1.1 -> 2.2 반대 순으로  수행 굵은선으로 표기

## **5.2 연쇄법칙**

- z = t^2 
- t = x + y
- az/ax = az/dt*at/ax 로 표기 
- az/ax = 2t*1 = 2(x+y)*1로 나타낼 수 있음


그림 5-7

- 역전파로 진행시 z -> az/az(derivative) -> az/az*az/at = az/at -> az/az*az/at*at/ax = az/ax x에 대한 z의 미분 


그림 5-8 

## **5.3 역전파**


- 덧셈 노드 역전파 : 똑같은 값을 전달 
- 곱셉 노드 역전파 : x와 y 값을 바꿔줌

In [ ]:
def backward(dout):  # 덧셈 역전파 
    dx = dout * 1 
    dy = dout * 1

In [ ]:
def backward(dout):   # 곱셈 역전파 
    dx = dout* self.y # x와 y를 바꿔준다 
    dy = dout* self.x 

- 10과 5가 곱해져서 50이 형성되지만 역전파로 진행했을 때 
- 50에 1.3이라는 전류가 흐를 때 
- 5에는 10 *1.3 = 13이라는 전류가 흐름 
- 10에는 1.3*5 = 6.5 전류가 흐름

## **5.5** 활성화 함수 계층 구현하기

- y = x (x > 0) , 0 (x <= 0)
- ay = 1 (x > 0), 0 (x <= 0)

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx   

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## **5.6 Affine/Softmax 계층 구현**

- np.dot(x, w) = 0
- (2, ) dot w (2, 3) = (3, ) 
- 앞에 2 -> 2 3 -> 3 매칭 
- 순전파 때 수행하는 곱을 어파인 변환이라고 칭함 


- a1 -> softmax y1 -> cross entropy error t1 -> L이 나오는 경우
- 역전파 경우 y1 - t1 값을 반환 
- (0, 1, 0) 정답 레이블일 때 
- (0.3, 0.2, 0.5) softmax 레이블 
- 정답률 20%가 되어 신경망 인식에 어려움 
- 역전파 경우 (0.3, -0.8, 0.5) 큰 오차를 통해 신경망이 인식


## **5.7 구현** 

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads